In [9]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torchsummary
import scripts.models as models
from scripts.models import StormFire

In [6]:
model = models.StormSqueezeNet()

model.to('cuda')
torchsummary.summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 111, 111]           1,792
         LeakyReLU-2         [-1, 64, 111, 111]               0
         MaxPool2d-3           [-1, 64, 55, 55]               0
            Conv2d-4           [-1, 16, 55, 55]           1,040
         LeakyReLU-5           [-1, 16, 55, 55]               0
       BatchNorm2d-6           [-1, 16, 55, 55]              32
            Conv2d-7           [-1, 64, 55, 55]           1,088
         LeakyReLU-8           [-1, 64, 55, 55]               0
       BatchNorm2d-9           [-1, 64, 55, 55]             128
           Conv2d-10           [-1, 64, 55, 55]           9,280
        LeakyReLU-11           [-1, 64, 55, 55]               0
      BatchNorm2d-12           [-1, 64, 55, 55]             128
        StormFire-13          [-1, 128, 55, 55]               0
           Conv2d-14           [-1, 16,

In [20]:

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.decode = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            StormFire(256, 64, 128,128),
            nn.Upsample(scale_factor=2),
            StormFire(128, 32, 64,64),
            nn.Upsample(scale_factor=2),
            StormFire(64, 16, 32,32),
            nn.Upsample(scale_factor=2),
            StormFire(32, 8, 16,16),
            nn.Conv2d(16, 3, kernel_size=3, padding=1),  # Output 3 channels for RGB
            nn.Tanh(),  # Output values between -1 and 1
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.decode(x)


           
        
class StormColorModel(nn.Module):
    # This net is a custom version of SqueezeNet: we replaced the Fire module with our custom StormFire module,
    # modified the final convolutional layer into a fully connected layer
    # we used leaky ReLU instead of ReLU
    # we added BatchNorm after each convolutional layer
    # we modified the number of fire modules and the number of filters in each fire module

    def __init__(self, num_classes: int = 251, dropout: float = 0.5) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.features = nn.Sequential(
            
            nn.Conv2d(1, 64, kernel_size=3, stride=2),
            nn.LeakyReLU(inplace=True),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(64, 16, 64, 64),
            StormFire(128, 16, 64, 64),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(128, 32, 128, 128),
            StormFire(256, 32, 128, 128),

            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            StormFire(256, 48, 192, 192),
            StormFire(384, 64, 256, 256),
            
        )

        self.decoder = Decoder()

        
       
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x  = self.features(x)
        print(x.size())
        x = self.decoder(x)
        print(x.size())
        return x




In [21]:
model = StormColorModel()
model.to('cuda')
torchsummary.summary(model, (1, 224, 224))

torch.Size([2, 512, 13, 13])


RuntimeError: Given groups=1, weight of size [32, 128, 1, 1], expected input[2, 256, 52, 52] to have 128 channels, but got 256 channels instead